In [1]:
import os, sys
import torch
sys.path.append('../../')
from IPython.display import display
from models.core.diffusion.pipe import Pipe
from models.core.diffusion.custom_pipeline import Generator4Embeds
from models.core.diffusion.diffusion_prior import DiffusionPriorUNet
from utils.data_modules.diffusion_embedding import DiffusionEmbeddingDataModule

In [2]:
data_module = DiffusionEmbeddingDataModule(
    eeg_embeddings_file="/workspace/eeg-image-decoding/data/all-joined-1/eeg/super-res-embeddings/650ms-250Hz/subj01_session1_eeg_embeddings.npy",
    subject=1,
    session=1,
    batch_size=1024,
    num_workers=4,
    val_split=0.1,
    test='default'
)

data_module.setup()
train_loader = data_module.train_dataloader()
test_loader = data_module.test_dataloader()

In [3]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [4]:
diffusion_prior = DiffusionPriorUNet(cond_dim=1024, dropout=0.1)
pipe = Pipe(diffusion_prior, device=device)

In [5]:
model_name = 'diffusion_prior' # 'diffusion_prior_vice_pre_imagenet' or 'diffusion_prior_vice_pre'
pipe.train(train_loader, num_epochs=150, learning_rate=1e-3)

epoch: 0, loss: 1.2798957029978435


/venv/main/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:182: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


epoch: 1, loss: 1.275378902753194
epoch: 2, loss: 1.2651424805323284
epoch: 3, loss: 1.2532401879628499
epoch: 4, loss: 1.2374232212702434
epoch: 5, loss: 1.2229764064153035
epoch: 6, loss: 1.2061510880788167
epoch: 7, loss: 1.1936326026916504
epoch: 8, loss: 1.1783708731333415
epoch: 9, loss: 1.164958079655965
epoch: 10, loss: 1.154543360074361
epoch: 11, loss: 1.1408860286076863
epoch: 12, loss: 1.1274080276489258
epoch: 13, loss: 1.1133510669072468
epoch: 14, loss: 1.1016571124394734
epoch: 15, loss: 1.0875865618387859
epoch: 16, loss: 1.0727136135101318
epoch: 17, loss: 1.0644642114639282
epoch: 18, loss: 1.047912319501241
epoch: 19, loss: 1.0401289065678914
epoch: 20, loss: 1.026387333869934
epoch: 21, loss: 1.0200438896814983
epoch: 22, loss: 1.0053085883458455
epoch: 23, loss: 0.9961893955866495
epoch: 24, loss: 0.986091693242391
epoch: 25, loss: 0.9732268850008646
epoch: 26, loss: 0.9615465005238851
epoch: 27, loss: 0.9512578050295512
epoch: 28, loss: 0.9387088815371195
epoch: 

In [6]:
save_path = f"/workspace/eeg-image-decoding/code/models/check_points/diffusion_prior/subj01_session1.pt"

# Save the trained diffusion prior
os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save(pipe.diffusion_prior.state_dict(), save_path)
print(f"Diffusion prior saved to {save_path}")

Diffusion prior saved to /workspace/eeg-image-decoding/code/models/check_points/diffusion_prior/subj01_session1.pt
